In [1]:
import torch
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

In [2]:
# _MODEL_PATH = "/mnt/nvme/MODELS/EMBEDINGS/e5-mistral-7b-instruct/"
_MODEL_PATH = "/mnt/nvme/MODELS/EMBEDINGS/UAE-Large-V1/"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(_MODEL_PATH, device="cuda:1")
model = AutoModel.from_pretrained(
    _MODEL_PATH,
    device_map="cuda:1",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

In [4]:
# def last_token_pool(
#     last_hidden_states: Tensor,
#     attention_mask: Tensor
#     ) -> Tensor:
    
#     left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
#     if left_padding:
#         return last_hidden_states[:, -1]
#     else:
#         sequence_lengths = attention_mask.sum(dim=1) - 1
#         batch_size = last_hidden_states.shape[0]
#         return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


In [5]:
# def get_detailed_instruct(task_description: str, query: str) -> str:
#     return f'Instruct: {task_description}\nQuery: {query}'


In [6]:
# Each query must come with a one-sentence instruction that describes the task
# task = 'Given a web search query, retrieve relevant passages that answer the query'
# input_texts = [
#     get_detailed_instruct(task, 'how much protein should a female eat'),
    
#     get_detailed_instruct(task, 'summit define'),
    
#     "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    
#     "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."
# ]

In [8]:
# max_length = 4096
# Tokenize the input texts
# batch_dict = tokenizer(input_texts, max_length=max_length - 1, return_attention_mask=False, padding=False, truncation=True)
# batch_dict = tokenizer(input_texts, return_attention_mask=False, padding=False, truncation=True)
# batch_dict

{'input_ids': [[101, 16021, 18300, 1024, 2445, 1037, 4773, 3945, 23032, 1010, 12850, 7882, 13768, 2008, 3437, 1996, 23032, 23032, 1024, 2129, 2172, 5250, 2323, 1037, 2931, 4521, 102], [101, 16021, 18300, 1024, 2445, 1037, 4773, 3945, 23032, 1010, 12850, 7882, 13768, 2008, 3437, 1996, 23032, 23032, 1024, 6465, 9375, 102], [101, 2004, 1037, 2236, 5009, 4179, 1010, 1996, 26629, 1005, 1055, 2779, 9095, 1997, 5250, 2005, 2308, 5535, 2539, 2000, 3963, 2003, 4805, 20372, 2566, 2154, 1012, 2021, 1010, 2004, 2017, 2064, 2156, 2013, 2023, 3673, 1010, 2017, 1005, 2222, 2342, 2000, 3623, 2008, 2065, 2017, 1005, 2128, 8074, 2030, 2731, 2005, 1037, 8589, 1012, 4638, 2041, 1996, 3673, 2917, 2000, 2156, 2129, 2172, 5250, 2017, 2323, 2022, 5983, 2169, 2154, 1012, 102], [101, 6210, 1997, 6465, 2005, 2394, 2653, 26262, 1012, 1024, 1015, 1996, 3284, 2391, 1997, 1037, 3137, 1024, 1996, 2327, 1997, 1037, 3137, 1012, 1024, 1016, 1996, 3284, 2504, 1012, 1024, 1017, 1037, 3116, 2030, 2186, 1997, 6295, 2090, 19

In [10]:
# append eos_token_id to every input_ids
# batch_dict['input_ids'] = [input_ids + [tokenizer.eos_token_id] for input_ids in batch_dict['input_ids']]
# batch_dict = tokenizer.pad(batch_dict, padding=True, return_attention_mask=True, return_tensors='pt')
# batch_dict = {k: v.to(model.device) for k, v in batch_dict.items()}

In [19]:
# outputs = model(**batch_dict)

In [22]:
# embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
# embeddings.shape

torch.Size([4, 4096])

In [23]:
# normalize embeddings
# embeddings = F.normalize(embeddings, p=2, dim=1)
# scores = (embeddings[:2] @ embeddings[2:].T) * 100
# print(scores.tolist())

[[82.9375, 47.96875], [46.9375, 81.8125]]


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
text1="""
USER: Hello Jarvis, how much would it cost to install new carpeting on my house's stairs? The steps are three feet broad and ten feet long. I would want to utilise a $5 per square foot carpet. 
ASSISTANT: Alright, Mr. I'll figure that out for you. <calculator> 3 * 10 30 <calculator> 30 * 5 150 </calculator>There would be a $150 overall cost.</s> Do you need my assistance with anything else right now?
"""

text2="""
USER: Hello Jarvis, how much would it cost to install new carpeting on my house's stairs? The steps are three feet broad and ten feet long. I would want to utilize a $5 per square foot carpet. 
ASSISTANT: Alright, Mr. I'll figure that out for you. <calculator> 3 * 10 30 <calculator> 30 * 5 150 </calculator>There would be a $150 overall cost.</s> Do you need my assistance with anything else right now?
"""
# text2 = input_texts[-1]

In [13]:
text1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
text1 = {k: v.to(model.device) for k, v in text1.items()}
text2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)
text2 = {k: v.to(model.device) for k, v in text2.items()}

In [14]:
text1['input_ids'].shape

torch.Size([1, 113])

In [15]:
# text1_embeddings = last_token_pool(model(**text1).last_hidden_state, text1['attention_mask'])
# text2_embeddings = last_token_pool(model(**text2).last_hidden_state, text2['attention_mask'])

text1_embeddings = model(**text1).last_hidden_state[:, -1]
text2_embeddings = model(**text2).last_hidden_state[:, -1]

In [16]:
text1_embeddings.shape, text2_embeddings.shape

(torch.Size([1, 1024]), torch.Size([1, 1024]))

In [17]:
cosine_similarity(text1_embeddings.cpu().detach().numpy(), text2_embeddings.cpu().detach().numpy())

array([[0.99798365]])

In [134]:
cosine(text1_embeddings.squeeze().cpu().detach().numpy(), text2_embeddings.squeeze().cpu().detach().numpy())

0.0009765625

In [84]:
text1_embeddings.cpu().detach().numpy()

array([[-1.724 ,  0.5835, -1.649 , ...,  0.2216, -2.031 ,  2.676 ]],
      dtype=float16)

In [85]:
text2_embeddings.cpu().detach().numpy()

array([[-1.724 ,  0.5835, -1.649 , ...,  0.2216, -2.031 ,  2.676 ]],
      dtype=float16)